# Úvod do regrese a knihovny ScikitLearn

* ScikitLearn je jednoduchá knihovna napsaná v Pythonu, která implementuje základní algoritmy strojového učení.
* Je postavená na knihovnách NumPy a SciPy
* Opensource pod licencí BSD
* Dokumentace: [http://scikit-learn.org/stable/documentation.html](http://scikit-learn.org/stable/documentation.html)

Pro seznámení s regresí v knihovně ScikitLearn budeme pracovat s datasetem cen nemovitostí. Tento dataset původně pochází z [www.zillow.com](https://www.zillow.com/research/data/), ale je zjednodušen a upraven na české prostředí.

## Načteme trénovací a testovací data do DataFramu Pandas.

* **Stav** je stav budovy na škále 1 až 10, kde 1 je nejhorší a 10 nejlepší.
* **plocha** je obytná plocha budovy v metrech čtverečních
* **rok_stavby** je rok výstavby domu
* **rok_prodeje** je rok, ve kterém byl dům prodaný
* **cena** je cena, za kterou se dům prodal 

In [1]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/mlcollege/ai-academy/main/06-Regrese/data/ceny_domu.csv", sep=',')
data.head()

,kvalita,plocha,datum_stavby,datum_prodeje,cena
0,7,158,2003,2008,5.21
1,6,117,1976,2007,4.54
2,7,165,2001,2008,5.59
3,7,159,1915,2006,3.50
4,8,204,2000,2008,6.25


## Rozdělíme si data na trénovací a testovací

In [2]:
from sklearn.model_selection import train_test_split

input_features = ['kvalita', 'plocha', 'datum_stavby',	'datum_prodeje']
target_feature = 'cena'

X_train, X_test, y_train, y_test = train_test_split(data[input_features], data[target_feature], test_size=0.1)
print('Velikost trenovaci mnoziny: {}'.format(len(X_train)))
print('Velikost testovaci mnoziny: {}'.format(len(X_test)))

Velikost trenovaci mnoziny: 1314
Velikost testovaci mnoziny: 146


## Trénování a predikce

Nyní si vytvoříme instanci modelu lineární regrese, model natrénujeme a vypíšeme koeficienty.


In [3]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
print (model.coef_) #koeficienty vstupnich atributu
print (model.intercept_) #konstantni posuv linearni kombinace

[0.63131642 0.01684869 0.01266083 0.00285111]
-32.372594128804266


Natrénovaný model můžeme použít k predikcím pro nová (testovací) data.

In [4]:
y_pred = model.predict(X_test)

In [5]:
import numpy as np
result =  pd.DataFrame(zip(y_test, np.around(y_pred, decimals=2)),
               columns =['skutecna_cena', 'predikce'])
result.head()

,skutecna_cena,predikce
0,3.28,3.13
1,5.91,6.19
2,5.19,5.24
3,3.25,2.81
4,3.52,3.09


## Uložení modelu na disk a opětovné načtení

Nejprve uložíme celý objekt model na disk do adresáře '/tmp/model.pkl'.

In [6]:
import pickle

with open('/tmp/model.pkl', 'wb') as f:
    pickle.dump(model, f)

Nyní ho z disku opět načteme a může opět používat.


In [7]:
with open('/tmp/model.pkl', 'rb') as f:
    model = pickle.load(f)

Můžeme jednoduše provést predikci i pro jeden smyšlený dům.

In [8]:
cena = model.predict([[8, 140, 2003, 2020]])
print ("Odhadovaná cena domu je: {}".format(cena[0]))

Odhadovaná cena domu je: 6.155640385876779


## Vyhodnocení kvality modelu

Nakonec určíme celkovou absolutní a kvadratickou chybu predikce na testovacím datasetu. Kvadratickou chybu pro snazší interpretaci odmocníme a dostaneme tak ukazatel, který se nazývá RMSE (Root Mean Squared Error).

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print ("MAE: {}".format(mean_absolute_error(y_test, y_pred)))
print ("RMSE: {}".format(mean_squared_error(y_test, y_pred, squared=False)))

MAE: 0.5963439841858947
RMSE: 0.763338734407555
